
<div style="
    background-color: #f7f7f7;
    background-image: url(''), url('') ;
    background-position: left bottom, right top;
    background-repeat: no-repeat,  no-repeat;
    background-size: auto 60px, auto 160px;
    border-radius: 5px;
    box-shadow: 0px 3px 1px -2px rgba(0, 0, 0, 0.2), 0px 2px 2px 0px rgba(0, 0, 0, 0.14), 0px 1px 5px 0px rgba(0,0,0,.12);">

<h1 style="
    color: #2a4cdf;
    font-style: normal;
    font-size: 2.25rem;
    line-height: 1.4em;
    font-weight: 600;
    padding: 30px 200px 0px 30px;"> 
        Query the perovskite ion database in NOMAD</h1>

<p style="
    line-height: 1.4em;
    padding: 30px 200px 0px 30px;">
        This notebook demonstrates how query NOMAD for all, as well as for specific perovskite ions. The notebook also shows how to validate the ions against the <a href="https://github.com/Jesperkemist/Perovskite_composition/blob/main/ion_schema.json" target="_blank">JSON Schema</a>

<p style="font-size: 1.25em; font-style: italic; padding: 5px 200px 30px 30px;">
    José A. Márquez and Hampus Näsström
</p>
</div>

### Basic imports

In [1]:
import json

import requests

base_url = 'http://nomad-lab.eu/prod/v1/api/v1'

### Query for an A site ion with abbreviation "MA"

In [2]:
response = requests.post(
    f'{base_url}/entries/archive/query',
    json={
        'owner': 'visible',
        'query': {
            'data.abbreviation#perovskite_solar_cell_database.composition.PerovskiteAIon:any': [
                'MA'
            ]
        },
        'pagination': {'page_size': 1},
    },
)
response_json = response.json()
ion_data = response_json['data'][0]['archive']['data']
print(json.dumps(ion_data, indent=2))

{
  "m_def": "perovskite_solar_cell_database.composition.PerovskiteAIon",
  "name": "MA perovskite ion",
  "datetime": "2024-12-18T14:46:39.211148+00:00",
  "lab_id": "perovskite_ion_MA",
  "common_name": "Methylammonium",
  "molecular_formula": "CH6N+",
  "smiles": "C[NH3+]",
  "iupac_name": "methylazanium",
  "cas_number": "17000-00-9",
  "abbreviation": "MA",
  "source_compound_molecular_formula": "CH5N",
  "source_compound_smiles": "CN",
  "source_compound_iupac_name": "methanamine",
  "source_compound_cas_number": "74-89-5",
  "elemental_composition": [
    {
      "element": "C",
      "atomic_fraction": 0.125,
      "mass_fraction": 0.3745730552651735
    },
    {
      "element": "H",
      "atomic_fraction": 0.75,
      "mass_fraction": 0.1886054095051807
    },
    {
      "element": "N",
      "atomic_fraction": 0.125,
      "mass_fraction": 0.4368215352296457
    }
  ],
  "pure_substance": {
    "name": "Methylammonium",
    "iupac_name": "methylazanium",
    "molecular_for

### Validate response against JSON Schema

In [3]:
from jsonschema import ValidationError, validate

schema_id = (
    'https://raw.githubusercontent.com/Jesperkemist/'
    'Perovskite_composition/v1.0.0/ion_schema.json'
)

# Load the JSON schema from the URL
schema_response = requests.get(schema_id)
schema = schema_response.json()

# Validate ion_data against the schema
try:
    validate(instance=ion_data, schema=schema)
    print(f'ion_data is valid against the {schema_id} schema.')
except ValidationError as e:
    print('Validation error:', e.message)

ion_data is valid against the https://raw.githubusercontent.com/Jesperkemist/Perovskite_composition/v1.0.0/ion_schema.json schema.


### Query all ions in the database

In [4]:
json_body = {
    'owner': 'visible',
    'query': {
        'results.eln.sections:any': [
            'PerovskiteAIon',
            'PerovskiteBIon',
            'PerovskiteXIon',
        ],
    },
    'pagination': {
        'page_size': 10,
    },
}

all_ion_data = {}

while len(all_ion_data) < 500:
    response = requests.post(f'{base_url}/entries/archive/query', json=json_body)
    response_json = response.json()

    for entry in response_json['data']:
        abbreviation = entry['archive']['data']['abbreviation']
        if abbreviation in all_ion_data:
            print(f'Duplicate entry found for abbreviation: {abbreviation}')
        all_ion_data[abbreviation] = entry['archive']['data']

    next_value = response_json['pagination'].get('next_page_after_value')
    if not next_value:
        break
    json_body['pagination']['page_after_value'] = next_value

print(f'Retrieved {len(all_ion_data)} ion entries.')

Retrieved 332 ion entries.


In [5]:
print(json.dumps(all_ion_data['MA'], indent=2))

{
  "m_def": "perovskite_solar_cell_database.composition.PerovskiteAIon",
  "name": "MA perovskite ion",
  "datetime": "2024-12-18T14:46:39.211148+00:00",
  "lab_id": "perovskite_ion_MA",
  "common_name": "Methylammonium",
  "molecular_formula": "CH6N+",
  "smiles": "C[NH3+]",
  "iupac_name": "methylazanium",
  "cas_number": "17000-00-9",
  "abbreviation": "MA",
  "source_compound_molecular_formula": "CH5N",
  "source_compound_smiles": "CN",
  "source_compound_iupac_name": "methanamine",
  "source_compound_cas_number": "74-89-5",
  "elemental_composition": [
    {
      "element": "C",
      "atomic_fraction": 0.125,
      "mass_fraction": 0.3745730552651735
    },
    {
      "element": "H",
      "atomic_fraction": 0.75,
      "mass_fraction": 0.1886054095051807
    },
    {
      "element": "N",
      "atomic_fraction": 0.125,
      "mass_fraction": 0.4368215352296457
    }
  ],
  "pure_substance": {
    "name": "Methylammonium",
    "iupac_name": "methylazanium",
    "molecular_for